# Тестирование

[Levels of testing](https://softwaretestingfundamentals.com/software-testing-levels/):

- Unit Testing: 	A level of the software testing process where individual units of a software are tested. The purpose is to validate that each unit of the software performs as designed.

- Integration Testing: 	A level of the software testing process where individual units are combined and tested as a group. The purpose of this level of testing is to expose faults in the interaction between integrated units.

- System Testing: 	A level of the software testing process where a complete, integrated system is tested. The purpose of this test is to evaluate the system’s compliance with the specified requirements.

- Acceptance Testing: 	A level of the software testing process where a system is tested for acceptability. The purpose of this test is to evaluate the system’s compliance with the business requirements and assess whether it is acceptable for delivery.

Сегодня мы концентрируемся на первом уровне -- юнит тестах. И конкретно на стандартной имплементации с помощью модуля [unittest](https://docs.python.org/3/library/unittest.html)

## Doctest

In [15]:
%%writefile gaf.py

def gaf(length=1, end=''):
    '''
    Гавкнуть длиной length с end в конце
    Параметры необязательные
    
    >>> gaf()
    'Gaf'
    
    >>> gaf(3)
    'Gaaaf'
    
    >>> gaf(4, '!')
    'Gaaaaf!'
    
    '''
    
    
    return 'G' + 'a' * length + 'f' + end

Overwriting gaf.py


In [16]:
! python3 -m doctest gaf.py

## Unittest

### Тестируем отдельные функции

In [67]:
%%writefile calculations.py

def add(x: float, y: float) -> float:
    return x + y

def divide(x: float, y: float) -> float:
    
    try:
        res = x / y
        
    except ZeroDivisionError as e:
        raise ValueError(e)
        res = 0
    
    return res

Overwriting calculations.py


In [72]:
%%writefile test_calculations.py

import unittest

import calculations


class TestCase(unittest.TestCase):
    
    def test_add(self):
        
        test_cases = [(4, 4), (10, 15), (-10, 10), (0.1, 0.2)]
        test_answers = [8, 25, 0, 0.3]
        
        for inputs, answer in zip(test_cases, test_answers):
            result = calculations.add(*inputs)
            self.assertAlmostEqual(result, answer)
#             self.assertEqual(result, answer)
    
    
    def test_divide(self):
        test_cases = [(10, 1), (10, 2), (5, 2)]
        test_answers = [10, 5, 2.5]
        
        for inputs, answer in zip(test_cases, test_answers):
            result = calculations.divide(*inputs)
            self.assertAlmostEqual(result, answer)
        
        with self.assertRaises(ValueError):
            res = calculations.divide(1, 0)
        
if __name__ == '__main__':
    unittest.main()

Overwriting test_calculations.py


In [73]:
! python3 -m unittest -v test_calculations.py

test_add (test_calculations.TestCase) ... ok
test_divide (test_calculations.TestCase) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


### Тестируем класс

In [83]:
%%writefile student.py

class Student:
    
    coeff_scholarship = 2
    
    def __init__(self, first_name, second_name, year, scholarship):
        self.first_name = first_name
        self.second_name = second_name
        self.year = year
        self.scholarship = scholarship
        
    @property
    def email(self):
        return f'{self.first_name.lower()[0]}{self.second_name.lower()}@edu.hse.ru'
    
    def increase_scholarship(self):
        self.scholarship = int(self.scholarship * self.coeff_scholarship)

Overwriting student.py


In [98]:
%%writefile test_student.py

import unittest

from student import Student

class TestStudent(unittest.TestCase):
    
    def setUp(self):
        '''
        Выполняется перед тестами
        '''
        
        self.default_name = 'Denis'
        self.default_surname = 'Belyakov'
        self.default_year = 2015
        self.default_scholar = 1
        
        self.t_st = Student(
            self.default_name,
            self.default_surname,
            self.default_year,
            self.default_scholar,
        )
    
    def tearDown(self):
        '''
        Выполняется, когда все тесты отработали
        '''
        
        pass
    
    def test_email(self):
        self.assertEqual(self.t_st.email, 'dbelyakov@edu.hse.ru')
        
        self.t_st.first_name = 'Kirill'
        self.assertEqual(self.t_st.email, 'kbelyakov@edu.hse.ru')
        
        self.t_st.first_name = self.default_name  # возвращаем атрибуты к дефолтным значениям
    
    def test_increase_scholarship(self):
        self.t_st.increase_scholarship()
        self.assertGreaterEqual(self.t_st.scholarship, self.default_scholar)
        
        self.t_st.scholarship = self.default_scholar
    
    
if __name__ == '__main__':
    unittest.main()

Overwriting test_student.py


In [99]:
! python3 -m unittest -v test_student.py

test_email (test_student.TestStudent) ... ok
test_increase_scholarship (test_student.TestStudent) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


### Тестируем Балабобу с прошлого семинара

Для того, чтобы протестировать функции получения ответа от Яндекса, нам надо их вынести в отдельный файл.

Или переписать исходный файл с кодом бота, завернув логику его запуска в 

```python 
if __name__ = '__main__':
    bot = ...
    ...
```

In [123]:
%%writefile balaboba.py

import time
import requests
import json


DAY2MONTH_RU = {
    1: 'Январь',
    2: 'Февраль',
    3: 'Март',
    4: 'Апрель',
    5: 'Май',
    6: 'Июнь',
    7: 'Июль',
    8: 'Август',
    9: 'Сентябрь',
    10: 'Октябрь',
    11: 'Ноябрь',
    12: 'Декабрь',
}

DAY2MONTH_EN = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December",
}

DEFAULT_PHRASE = 'There is nothing.'


def add_date_remainder(day_of_month: int):
    '''
    Add day_of_month remainder for English language
    (contributed by strudent)
    '''
    return f'the {day_of_month}{"st" if (remainder := day_of_month % 10) == 1 and day_of_month != 11 else "nd" if remainder == 2 and day_of_month != 12 else "rd" if remainder == 3 and day_of_month != 13 else "th"}'


def generate_data_phrase(phrase: str='Armenia', lang: str='en'):
    time_struct = time.localtime()
        
    if lang == 'ru':    
        date_str = f'{phrase}, {time_struct.tm_mday} число, {DAY2MONTH_RU[time_struct.tm_mon]}, {time_struct.tm_hour:02}:{time_struct.tm_min:02}:{time_struct.tm_sec:02}.'
    else:
        date_str = f'{phrase}, {add_date_remainder(time_struct.tm_mday)} of {DAY2MONTH_EN[time_struct.tm_mon]}, {time_struct.tm_hour:02}:{time_struct.tm_min:02}:{time_struct.tm_sec:02}.'
                
    return date_str


def ask_balaboba(phrase: str = 'Russia',
                 lang: str = 'en',
                 url: str = 'https://yandex.ru/lab/api/yalm/text3',
                 delay: int = 0,
                 intro: int = 28):
    time.sleep(delay)

    query = generate_data_phrase(phrase=phrase, lang=lang)

    headers = {
        'Content-Type': 'application/json',
        'Origin': 'https://yandex.com',
        'Referer': 'https://yandex.com',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0',
    }
    
    if lang == 'ru':
        intro = 24
        
    payload = json.dumps({
        "query": query,
        "intro": intro,  # 28
        "filter": 1,
    })

    response = requests.request('POST', url, headers=headers, data=payload)

    if response.ok:
        text = response.json()['text']
        text = DEFAULT_PHRASE if not text else text

        return f'{query} {text}'

    else:
        return f'No story. Status code {response.status_code}'

Overwriting balaboba.py


Функция похода по api зависит от того, как работает Яндекс, а мы хотим проверять именно наш код. Можно использовать механизм "моков" и определить ("замокать") какие-то значения для функции `requests.request`, как будто мы ее правда вызвали.

In [124]:
%%writefile test_balaboba.py

import unittest
import time

import balaboba

from unittest.mock import patch, Mock

class TestBalaboba(unittest.TestCase):
    
    def test_ask_balaboba(self):
        with patch('balaboba.requests.request') as mocked_request:
            # задаем выходные значения для requests.request для случая ошибки от сервиса Яндекса
            mocked_request.return_value.ok = False
            mocked_request.return_value.status_code = 403
            expected_value = 'No story. Status code 403'
            
            actual_value = balaboba.ask_balaboba()
            mocked_request.assert_called()
            self.assertEqual(actual_value, expected_value)
            
            # задаем выходные значения для функции requests.request для случая корректного поведения
            mocked_request.return_value = Mock( 
                status_code=200,
                ok=True,
                json=lambda : {'text': 'Some short story'}
            )
            
            start_time = time.time()  # считаем время для проверки корректности работы параметра delay
            actual_value = balaboba.ask_balaboba('Armenia', delay=1)
            total_time = time.time() - start_time

            self.assertIn('Armenia', actual_value)
            self.assertIn('Some short story', actual_value)
            self.assertGreater(total_time, 1)
            
    
if __name__ == '__main__':
    unittest.main()

Overwriting test_balaboba.py


In [125]:
! python3 -m unittest -v test_balaboba.py

test_ask_balaboba (test_balaboba.TestBalaboba) ... ok

----------------------------------------------------------------------
Ran 1 test in 1.002s

OK
